Pre-processing the ATAC seq data using the outputs from Cell Ranger ARC
we should have:
atac_fragments.tsv.gz
atac_peaks.bed
per_barcode_metrics.csv

In [34]:
import pycisTopic
pycisTopic.__version__

'2.0a0'

In [35]:
# Set up output structure
import os
out_dir = "outs"
os.makedirs(out_dir, exist_ok=True)

# Define fragments mapping for your sample(s)
fragments_dict = {
    "aspc1_1_untrimmed": "/scratcha/fmlab/blake01/yard/cellranger_arc/run_cellranger_arc_counts/aspc1_1_untrimmed/outs/atac_fragments.tsv.gz"
}


Getting pseudobulk profiles from cell annotations

In [36]:
# Import pandas first
import pandas as pd

In [37]:
import scanpy as sc

In [46]:
# load pre‐processed AnnData (with leiden clusters already computed) from rna seq pre-processing
adata = sc.read_h5ad("adata_processed.h5ad")

In [47]:
# 1a) Suppose your AnnData is called `adata` and you ran:
#     sc.tl.leiden(adata, key_added='leiden')
#     so your cell barcodes live in adata.obs_names
#     and the cluster labels in adata.obs['leiden'].

cell_data = pd.DataFrame({
    # index must be the exact barcodes in fragments.tsv.gz
    # if it has a suffix (e.g. "-1") make sure it's identical
    'barcode': adata.obs_names,   
    # this is the grouping variable for pseudobulk
    'leiden_cluster': adata.obs['leiden'].astype(str),
    # CELLRANGER ARC sample ID: name of the run folder containing fragments.tsv.gz
    'sample_id': 'aspc1_1_untrimmed'      
})
cell_data = cell_data.set_index('barcode')
cell_data.head()

,leiden_cluster,sample_id
barcode,,
AAACAGCCAAATTGCT-1,0,aspc1_1_untrimmed
AAACAGCCAATGCCCG-1,6,aspc1_1_untrimmed
AAACAGCCACACCAAC-1,4,aspc1_1_untrimmed
AAACAGCCACTTAGGC-1,3,aspc1_1_untrimmed
AAACAGCCATTGACAT-1,4,aspc1_1_untrimmed


Load Chromosome Sizes

In [48]:
chromsizes = pd.read_table(
    "http://hgdownload.cse.ucsc.edu/goldenPath/hg38/bigZips/hg38.chrom.sizes",
    header=None,
    names=["Chromosome","End"]
)
chromsizes.insert(1, "Start", 0)

Point to the fragments file:
cell ranger arc places the fragments.tsv.gz file here
<run_folder>/outs/atac_fragments.tsv.gz

In [49]:
from pathlib import Path

run_folder = Path("/mnt/scratcha/fmlab/blake01/yard/cellranger_arc/run_cellranger_arc_counts/aspc1_1_untrimmed/outs")
fragments_dict = {
    'aspc1_1_untrimmed': str(run_folder / "atac_fragments.tsv.gz")
}

Export Pseudobulk
This function will generate a fragments.tsv.gz and a bigwig file for each cell type defined by the variable parameter.

In [ ]:
from pycisTopic.pseudobulk_peak_calling import export_pseudobulk
import os

out_dir = "outs"
os.makedirs(os.path.join(out_dir, "consensus_peak_calling/pseudobulk_bed_files"), exist_ok=True)
os.makedirs(os.path.join(out_dir, "consensus_peak_calling/pseudobulk_bw_files"), exist_ok=True)

bw_paths, bed_paths = export_pseudobulk(
    input_data=cell_data,
    variable="leiden_cluster",
    sample_id_col="sample_id",
    chromsizes=chromsizes,
    bed_path=os.path.join(out_dir, "consensus_peak_calling/pseudobulk_bed_files"),
    bigwig_path=os.path.join(out_dir, "consensus_peak_calling/pseudobulk_bw_files"),
    path_to_fragments=fragments_dict,
    n_cpu=10,
    normalize_bigwig=True,
    temp_dir="/tmp",
    split_pattern=None  # or "-" if your barcodes include sample ID
)

2025-04-25 18:36:21,360 cisTopic     INFO     Splitting fragments by cell type.


Save the Paths! 

In [ ]:
pd.Series(bw_paths, name="bigwig").to_csv(f"{out_dir}/bw_paths.tsv", sep="\t")
pd.Series(bed_paths, name="bed").to_csv(f"{out_dir}/bed_paths.tsv", sep="\t")

Load QC metrics and pick high-quality cells

In [ ]:
# 1a) Read in the per‐barcode ATAC metrics
metrics = pd.read_csv("per_barcode_metrics.csv", index_col=0)

In [ ]:
# 1b) Define a simple QC filter, e.g. keep cells with ≥1,000 fragments & FRiP ≥20%
keep_cells = metrics.query("n_fragments >= 1000 and FRIP >= 0.2").index.tolist()

print(f"Keeping {len(keep_cells)} high‐quality ATAC cells")

Create a cisTopic object which is a peak x cell count matrix

In [ ]:
from pycisTopic.cistopic_classes import create_cistopic_object_from_csv

# This utility will read your peaks, your fragments file, 
# and count overlaps only for the barcodes in 'keep_cells'.

cst = create_cistopic_object_from_csv(
    csvfile="atac_fragments.tsv.gz",         # tab‐delimited with chr, start, end, barcode
    sep="\t",
    count_matrix=None,                        # we’ll generate it
    cell_column="barcode",
    peak_file="atac_peaks.bed",               # your peak definitions
    valid_cells=keep_cells,                   # only these barcodes
    format="BED",                             # fragments in BED format
    genome="hg38"
)

In [ ]:
print(cst)  # sanity check: shows #peaks × #cells

Filter low-information peaks and cells

In [ ]:
# 3a) Remove peaks seen in < 1% of cells
min_cells = int(0.01 * len(keep_cells))
cst.filter_peaks(min_cells=min_cells)

In [ ]:
# 3b) Optionally, remove cells with very few peaks
cst.filter_cells(min_counts=500)

In [ ]:
print(f"After filtering: {cst.n_peaks} peaks, {cst.n_cells} cells")